<a href="https://colab.research.google.com/github/jorgenotte/TiVA/blob/main/TiVA_FIGARO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Crear DataFrame con el mapeo de codigos nace y codigos geo
mapgeo=pd.read_excel('/content/drive/MyDrive/FIGARO/mapeo.xlsx','geo').set_index('cod')
mapnace=pd.read_excel('/content/drive/MyDrive/FIGARO/mapeo.xlsx','nace').set_index('cod')

# Convertir los DataFrame en diccionarios
mapgeo=dict(mapgeo.to_dict()['geo'])
mapnace=dict(mapnace.to_dict()['nace'])

In [10]:
# Leer el fichero en un dataframe [9363300 rows × 7 columns] [FORMATO LARGO]
ruta='/content/drive/MyDrive/FIGARO/flatfile_eu-ic-io_ind-by-ind_23ed_2021.csv'
df=pd.read_csv(ruta)

In [149]:
# Se crea la columna agrega
df['agrega'] = df['refArea'].map(mapgeo) + '_' + df['rowIi'].map(mapnace) + '*' + df['counterpartArea'].map(mapgeo) + '_' + df['colIi'].map(mapnace)
df_colapsado=pd.DataFrame(df.groupby(['agrega'])['obsValue'].sum()).reset_index()

In [150]:
# Se separa la columna agrega en las columnas origen y destino
df_colapsado['origen']=df_colapsado['agrega'].str.split('*').str[0]
df_colapsado['destino']=df_colapsado['agrega'].str.split('*').str[1]

In [151]:
df_colapsado

,agrega,obsValue,origen,destino
0,ES_A*ES_A,2928.305,ES_A,ES_A
1,ES_A*ES_BDE,66.757,ES_A,ES_BDE
2,ES_A*ES_C,24086.185,ES_A,ES_C
3,ES_A*ES_F,72.789,ES_A,ES_F
4,ES_A*ES_GHI,1349.627,ES_A,ES_GHI
...,...,...,...,...
1867,W2_OP_RES*RW_P3_S14,446721.992,W2_OP_RES,RW_P3_S14
1868,W2_OP_RES*RW_P3_S15,0.000,W2_OP_RES,RW_P3_S15
1869,W2_OP_RES*RW_P51G,0.000,W2_OP_RES,RW_P51G
1870,W2_OP_RES*RW_P5M,0.000,W2_OP_RES,RW_P5M


In [152]:
# Se definen las variables del VA
valores1 = ['D21X31', 'OP_NRES', 'OP_RES', 'D1', 'D29X39', 'B2A3G']

# Se definen las variables de la demanda final
valores2 = ['P3_S13', 'P3_S14', 'P3_S15', 'P51G', 'P5M']

# Expresión regular que coincide con cualquiera de los caracteres exactos especificados
patron1 = '|'.join(valores1)
patron2 = '|'.join(valores2)

In [153]:
# Se crean los siguientes df: (z=Consumos intermedios) / (tls1=T-S s/productos) / (va=Value added)
df_z_pre = df_colapsado[~df_colapsado['origen'].str.contains(patron1, regex=True) & ~df_colapsado['destino'].str.contains(patron2, regex=True) ].reset_index(drop=True)
df_tls1 = df_colapsado[(df_colapsado['origen'] == 'W2_D21X31') & ~df_colapsado['destino'].str.contains(patron2, regex=True)].reset_index(drop=True)
df_va_pre = df_colapsado[((df_colapsado['origen'] == 'W2_D1') | (df_colapsado['origen'] == 'W2_D29X39') | (df_colapsado['origen'] == 'W2_B2A3G')) & ~df_colapsado['destino'].str.contains(patron2, regex=True)].reset_index(drop=True)

In [154]:
df_z_pre

,agrega,obsValue,origen,destino
0,ES_A*ES_A,2928.305,ES_A,ES_A
1,ES_A*ES_BDE,66.757,ES_A,ES_BDE
2,ES_A*ES_C,24086.185,ES_A,ES_C
3,ES_A*ES_F,72.789,ES_A,ES_F
4,ES_A*ES_GHI,1349.627,ES_A,ES_GHI
...,...,...,...,...
1084,RW_RSTU*RW_K,34636.943,RW_RSTU,RW_K
1085,RW_RSTU*RW_L,17486.557,RW_RSTU,RW_L
1086,RW_RSTU*RW_MN,91192.087,RW_RSTU,RW_MN
1087,RW_RSTU*RW_OPQ,151367.220,RW_RSTU,RW_OPQ


In [155]:
# Se crea una nueva columna para conocer el total de CI
df_z_pre['agrega2'] = 'CI' + '*' + df_z_pre['destino']
df_z=pd.DataFrame(df_z_pre.groupby(['agrega2'])['obsValue'].sum()).reset_index()

# Se separa la columna agrega2 en las columnas origen y destino
df_z['origen']=df_z['agrega2'].str.split('*').str[0]
df_z['destino']=df_z['agrega2'].str.split('*').str[1]

In [156]:
# Se crea una nueva columna para conocer el total de VA
df_va_pre['agrega2'] = 'VA' + '*' + df_va_pre['destino']
df_va=pd.DataFrame(df_va_pre.groupby(['agrega2'])['obsValue'].sum()).reset_index()

# Se separa la columna agrega2 en las columnas origen y destino
df_va['origen']=df_va['agrega2'].str.split('*').str[0]
df_va['destino']=df_va['agrega2'].str.split('*').str[1]

In [157]:
# Se concatenan los df de CI, VA y T-S s/productos para conocer el total de X (Producción)
df_x_pre = pd.concat([df_z, df_va, df_tls1], ignore_index=True)

# Se crea una nueva columna para conocer el total de X (Producción)
df_x_pre['agrega2'] = 'X' + '*' + df_va_pre['destino']
df_x=pd.DataFrame(df_x_pre.groupby(['agrega2'])['obsValue'].sum()).reset_index()

# Se separa la columna agrega2 en las columnas origen y destino
df_x['origen']=df_x['agrega2'].str.split('*').str[0]
df_x['destino']=df_x['agrega2'].str.split('*').str[1]

In [158]:
# Se crean los siguientes df: (y=Demanda final) / (tls2=T-S s/productos) / (op=Purchases of non-residents in the domestic territory and direct purchase abroad by residents)
df_y = df_colapsado[~df_colapsado['origen'].str.contains(patron1, regex=True) & df_colapsado['destino'].str.contains(patron2, regex=True)].reset_index(drop=True)
df_tls2 = df_colapsado[(df_colapsado['origen'] == 'W2_D21X31') & df_colapsado['destino'].str.contains(patron2, regex=True)].reset_index(drop=True)
df_op = df_colapsado[((df_colapsado['origen'] == 'W2_OP_RES') | (df_colapsado['origen'] == 'W2_OP_NRES')) & df_colapsado['destino'].str.contains(patron2, regex=True)].reset_index(drop=True)

In [159]:
# Se crean los diferentes df en formato ancho
df_z_pre_tabla = df_z_pre.pivot_table(index="origen", columns="destino", values="obsValue")

df_z_tabla = df_z.pivot_table(index="origen", columns="destino", values="obsValue")

df_tls1_tabla = df_tls1.pivot_table(index="origen", columns="destino", values="obsValue")

df_va_tabla = df_va.pivot_table(index="origen", columns="destino", values="obsValue")

df_y_tabla = df_y.pivot_table(index="origen", columns="destino", values="obsValue")

df_tls2_tabla = df_tls2.pivot_table(index="origen", columns="destino", values="obsValue")

df_op_tabla = df_op.pivot_table(index="origen", columns="destino", values="obsValue")

df_x_tabla = df_x.pivot_table(index="origen", columns="destino", values="obsValue")

In [160]:
# Se extraen los valores numéricos de los diferentes df y se asignan a una matriz
matriz_z_pre = df_z_pre_tabla.values
matriz_z = df_z_tabla.values
matriz_tls1 = df_tls1_tabla.values
matriz_va = df_va_tabla.values
matriz_y = df_y_tabla.values
matriz_tls2 = df_tls2_tabla.values
matriz_op = df_op_tabla.values
matriz_x = df_x_tabla.values

In [161]:
# Value added to output ratio (VA/X)
np.seterr(divide='ignore', invalid='ignore')
matriz_ratio = np.where(matriz_x != 0, matriz_va / matriz_x, 0)

# Crear DataFrame y asignar nuevos índices
df_ratio = pd.DataFrame(matriz_ratio, columns=df_va_tabla.columns, index=df_va_tabla.index)
nuevo_indice = ['VA/X']
df_ratio.index = pd.Index(nuevo_indice, name='origen')

In [162]:
# Input coefficients (Z/X)
input_coeff = np.where(matriz_x != 0, matriz_z_pre / matriz_x, 0)

# Crear DataFrame y asignar nuevos índices
df_input_coeff = pd.DataFrame(input_coeff, columns=df_z_pre_tabla.columns, index=df_z_pre_tabla.index)

In [163]:
# Matriz identidad
num_filas = len(df_z_pre_tabla)
matriz_identidad = np.eye(num_filas)
df_matriz_identidad = pd.DataFrame(matriz_identidad, columns=df_z_pre_tabla.columns, index=df_z_pre_tabla.index)

# Diferencia entre matriz identidad y matriz de coeficientes técnicos
diferencia = matriz_identidad - input_coeff
df_diferencia = pd.DataFrame(diferencia, columns=df_z_pre_tabla.columns, index=df_z_pre_tabla.index)

# Matriz inversa de Leontief
inversa = np.linalg.inv(diferencia)
df_inversa = pd.DataFrame(inversa, columns=df_z_pre_tabla.columns, index=df_z_pre_tabla.index)

In [164]:
# Gross intermediate trade flows (GRTR_INT)

# Se divide las cadenas de texto que se encuentran en las columnas 'origen' y 'destino' en dos partes
df_z_pre[['origen2','x1']] = df_z_pre['origen'].str.split('_', n=1, expand=True)
df_z_pre[['destino2','x2']] = df_z_pre['destino'].str.split('_', n=1, expand=True)
df_z_pre = df_z_pre.drop(['x1','x2'], axis=1)

df_z_grtr = df_z_pre.copy()
df_z_grtr['obsValue2'] = np.where(df_z_grtr['origen2'] == df_z_grtr['destino2'], 0, df_z_grtr['obsValue'])
df_z_grtr['agrega2'] = df_z_grtr['origen'] + '*' + df_z_grtr['destino2']

df_z_grtr2=pd.DataFrame(df_z_grtr.groupby(['agrega2'])['obsValue2'].sum()).reset_index()
df_z_grtr2[['origen2','destino2']] = df_z_grtr2['agrega2'].str.split('*', n=1, expand=True)


In [165]:
df_z_grtr_table = df_z_grtr2.pivot_table(index="origen2", columns="destino2", values="obsValue2")
df_z_grtr_table

destino2,ES,FR,RW
origen2,,,
ES_A,0.000,1387.966,9530.994
ES_BDE,0.000,799.750,5195.943
ES_C,0.000,18769.316,102748.230
ES_F,0.000,152.697,1459.858
ES_GHI,0.000,9317.705,52734.549
ES_J,0.000,522.999,7080.433
ES_K,0.000,380.455,4202.726
ES_L,0.000,25.788,161.760
ES_MN,0.000,2179.499,16443.755


In [166]:
# Gross final trade flows (GRTR_FNL)

# Se divide las cadenas de texto que se encuentran en las columnas 'origen' y 'destino' en dos partes
df_y[['origen2','x1']] = df_y['origen'].str.split('_', n=1, expand=True)
df_y[['destino2','x2']] = df_y['destino'].str.split('_', n=1, expand=True)
df_y = df_y.drop(['x1','x2'], axis=1)

df_y_grtr = df_y.copy()
df_y_grtr['obsValue2'] = np.where(df_y_grtr['origen2'] == df_y_grtr['destino2'], 0, df_y_grtr['obsValue'])
df_y_grtr['agrega2'] = df_y_grtr['origen'] + '*' + df_y_grtr['destino2']

df_y_grtr2=pd.DataFrame(df_y_grtr.groupby(['agrega2'])['obsValue2'].sum()).reset_index()
df_y_grtr2[['origen2','destino2']] = df_y_grtr2['agrega2'].str.split('*', n=1, expand=True)

In [167]:
df_y_grtr_table = df_y_grtr2.pivot_table(index="origen2", columns="destino2", values="obsValue2")
df_y_grtr_table

destino2,ES,FR,RW
origen2,,,
ES_A,0.000,2694.248,6603.589
ES_BDE,0.000,48.542,448.048
ES_C,0.000,15933.519,77750.166
ES_F,0.000,35.184,859.416
ES_GHI,0.000,1513.329,19257.507
ES_J,0.000,868.468,6905.410
ES_K,0.000,164.310,2371.898
ES_L,0.000,5.643,119.740
ES_MN,0.000,553.652,4993.724


In [168]:
# Gross total trade flows (GRTR_TOTAL)

df_zy_grtr_table = df_z_grtr_table + df_y_grtr_table
df_zy_grtr_table

destino2,ES,FR,RW
origen2,,,
ES_A,0.000,4082.214,16134.583
ES_BDE,0.000,848.292,5643.991
ES_C,0.000,34702.835,180498.396
ES_F,0.000,187.881,2319.274
ES_GHI,0.000,10831.034,71992.056
ES_J,0.000,1391.467,13985.843
ES_K,0.000,544.765,6574.624
ES_L,0.000,31.431,281.500
ES_MN,0.000,2733.151,21437.479


In [169]:
ruta_excel = '/content/drive/MyDrive/FIGARO/Indicadores TiVA.xlsx'

with pd.ExcelWriter(ruta_excel, engine='openpyxl') as writer:
    # Guardar cada DataFrame en una hoja diferente
    df_z_pre_tabla.to_excel(writer, sheet_name='Matriz CI', index=True)
    df_y_tabla.to_excel(writer, sheet_name='Matriz DF', index=True)
    df_tls1_tabla.to_excel(writer, sheet_name='Impuestos CI', index=True)
    df_tls2_tabla.to_excel(writer, sheet_name='Impuestos DF', index=True)
    df_va_tabla.to_excel(writer, sheet_name='VA', index=True)
    df_op_tabla.to_excel(writer, sheet_name='OP_RES_NRES', index=True)
    df_x_tabla.to_excel(writer, sheet_name='Producción', index=True)
    df_ratio.to_excel(writer, sheet_name='Ratio VA to X', index=True)
    df_z_grtr_table.to_excel(writer, sheet_name='GITF', index=True)
    df_y_grtr_table.to_excel(writer, sheet_name='GFTF', index=True)
    df_zy_grtr_table.to_excel(writer, sheet_name='GRTR_TOTAL', index=True)
    df_input_coeff.to_excel(writer, sheet_name='Input coefficient', index=True)
    df_matriz_identidad.to_excel(writer, sheet_name='Identity matrix', index=True)
    df_diferencia.to_excel(writer, sheet_name='I-A', index=True)
    df_inversa.to_excel(writer, sheet_name='Leontief inverse', index=True)

### **STRUCTURAL INDICATORS - BASED ON THE VALUES IN ICIO TABLES**

In [170]:
# 4.1 Production (Gross output) [OPCION-1]

production = round(df_x.loc[df_x['destino'].str.contains('ES'), 'obsValue'].sum(), 1)
print("La producción de ES es:", production)

production1 = round(df_x.loc[df_x['destino'].str.contains('FR'), 'obsValue'].sum(), 1)
print("La producción de FR es:", production1)

production2 = round(df_x.loc[df_x['destino'].str.contains('RW'), 'obsValue'].sum(), 1)
print("La producción de RW es:", production2)

La producción de ES es: 2134504.7
La producción de FR es: 4433156.5
La producción de RW es: 156233689.4


In [171]:
# 4.1 Production (Gross output) [OPCION-2]

indices_x_ES = [i for i, col in enumerate(df_x_tabla.columns) if 'ES' in col]
indices_x_FR = [i for i, col in enumerate(df_x_tabla.columns) if 'FR' in col]
indices_x_RW = [i for i, col in enumerate(df_x_tabla.columns) if 'RW' in col]

In [172]:
suma_prod_ES = np.round((np.sum(matriz_x[:, indices_x_ES], axis=1)), decimals=1)
suma_prod_FR = np.round((np.sum(matriz_x[:, indices_x_FR], axis=1)), decimals=1)
suma_prod_RW = np.round((np.sum(matriz_x[:, indices_x_RW], axis=1)), decimals=1)

In [173]:
print("La producción de ES es:", suma_prod_ES)
print("La producción de FR es:", suma_prod_FR)
print("La producción de RW es:", suma_prod_RW)

La producción de ES es: [2134504.7]
La producción de FR es: [4433156.5]
La producción de RW es: [1.56233689e+08]


In [174]:
# 4.1 Production (Gross output) [OPCION-3]

dic_matriz_x = {}
for fila in df_x_tabla.index:
    for columna in df_x_tabla.columns:
        dic_matriz_x[(fila, columna)] = matriz_x[df_x_tabla.index.get_loc(fila), df_x_tabla.columns.get_loc(columna)]

In [175]:
print(dic_matriz_x)

{('X', 'ES_A'): 60815.787, ('X', 'ES_BDE'): 101384.83499999999, ('X', 'ES_C'): 488610.57599999994, ('X', 'ES_F'): 157166.00100000002, ('X', 'ES_GHI'): 476591.68, ('X', 'ES_J'): 91442.864, ('X', 'ES_K'): 81849.841, ('X', 'ES_L'): 149935.93800000002, ('X', 'ES_MN'): 173259.57499999998, ('X', 'ES_OPQ'): 283910.88300000003, ('X', 'ES_RSTU'): 69536.762, ('X', 'FR_A'): 97018.484, ('X', 'FR_BDE'): 203727.976, ('X', 'FR_C'): 778766.112, ('X', 'FR_F'): 323530.83900000004, ('X', 'FR_GHI'): 793006.7779999999, ('X', 'FR_J'): 246953.74, ('X', 'FR_K'): 249721.46899999998, ('X', 'FR_L'): 343346.36, ('X', 'FR_MN'): 617987.686, ('X', 'FR_OPQ'): 678542.7749999999, ('X', 'FR_RSTU'): 100554.289, ('X', 'RW_A'): 5885225.184, ('X', 'RW_BDE'): 8635386.45, ('X', 'RW_C'): 45289044.165, ('X', 'RW_F'): 11190932.622000001, ('X', 'RW_GHI'): 24450690.548, ('X', 'RW_J'): 6804830.438999999, ('X', 'RW_K'): 8488017.591, ('X', 'RW_L'): 9584095.854, ('X', 'RW_MN'): 11520293.967, ('X', 'RW_OPQ'): 21274950.867, ('X', 'RW_RS

In [176]:
# Se suman solo los valores donde la clave cumple cierta condición
suma_prod_ES = round(sum(valor for clave, valor in dic_matriz_x.items() if 'ES' in clave[1]),1)
print("La producción de ES es:", suma_prod_ES)

suma_prod_FR = round(sum(valor for clave, valor in dic_matriz_x.items() if 'FR' in clave[1]),1)
print("La producción de FR es:", suma_prod_FR)

suma_prod_RW = round(sum(valor for clave, valor in dic_matriz_x.items() if 'RW' in clave[1]),1)
print("La producción de RW es:", suma_prod_RW)

La producción de ES es: 2134504.7
La producción de FR es: 4433156.5
La producción de RW es: 156233689.4


In [177]:
# 4.2 Value added

value_added = round(df_va.loc[df_va['destino'].str.contains('ES'), 'obsValue'].sum(), 1)
print("El valor añadido de ES es:", value_added)

value_added1 = round(df_va.loc[df_va['destino'].str.contains('FR'), 'obsValue'].sum(), 1)
print("El valor añadido de FR es:", value_added1)

value_added2 = round(df_va.loc[df_va['destino'].str.contains('RW'), 'obsValue'].sum(), 1)
print("El valor añadido de RW es:", value_added2)

El valor añadido de ES es: 1090941.2
El valor añadido de FR es: 2216805.3
El valor añadido de RW es: 74148515.0


In [178]:
# 4.3 Value added as a % of production

df_ratio

destino,ES_A,ES_BDE,ES_C,ES_F,ES_GHI,ES_J,ES_K,ES_L,ES_MN,ES_OPQ,...,RW_BDE,RW_C,RW_F,RW_GHI,RW_J,RW_K,RW_L,RW_MN,RW_OPQ,RW_RSTU
origen,,,,,,,,,,,,,,,,,,,,,
VA/X,0.537515,0.327085,0.231397,0.427909,0.547757,0.478305,0.563614,0.862371,0.610014,0.738887,...,0.464359,0.279804,0.366408,0.522191,0.542635,0.584873,0.742609,0.543629,0.65193,0.582125


In [179]:
df_ratio_pre = pd.melt(df_ratio, var_name='destino', value_name='obsValue')

df_filtrado1 = df_ratio_pre[df_ratio_pre['destino'].str.contains('ES')]

maximos = df_filtrado1.sort_values(by='obsValue', ascending=False).head(1)
minimos = df_filtrado1.sort_values(by='obsValue', ascending=True).head(1)
destino_max = maximos.iloc[0]['destino']
obsValue_max = maximos.iloc[0]['obsValue']
destino_min = minimos.iloc[0]['destino']
obsValue_min = minimos.iloc[0]['obsValue']
print("La rama de actividad de ES que presenta mayor ratio es:", destino_max)
print("Siendo su valor:", round(obsValue_max, 4))
print("La rama de actividad de ES que presenta menor ratio es:", destino_min)
print("Siendo su valor:", round(obsValue_min,4))


La rama de actividad de ES que presenta mayor ratio es: ES_L
Siendo su valor: 0.8624
La rama de actividad de ES que presenta menor ratio es: ES_C
Siendo su valor: 0.2314


In [180]:
# 4.4 Gross exports

# Intermediate products
int_exports1 = round(df_z_grtr.loc[(df_z_grtr['origen2'] == 'ES') & (df_z_grtr['destino2'] == 'FR'), 'obsValue2'].sum(), 1)
print("Las exportaciones de productos intermedios para ES con destino FR son:", int_exports1)

int_exports2 = round(df_z_grtr.loc[(df_z_grtr['origen2'] == 'ES') & (df_z_grtr['destino2'] == 'RW'), 'obsValue2'].sum(), 1)
print("Las exportaciones de productos intermedios para ES con destino RW son:", int_exports2)

# Final products
final_exports1 = round(df_y_grtr.loc[(df_y_grtr['origen2'] == 'ES') & (df_y_grtr['destino2'] == 'FR'), 'obsValue2'].sum(), 1)
print("Las exportaciones de productos terminados para ES con destino FR son:", final_exports1)

final_exports2 = round(df_y_grtr.loc[(df_y_grtr['origen2'] == 'ES') & (df_y_grtr['destino2'] == 'RW'), 'obsValue2'].sum(), 1)
print("Las exportaciones de productos terminados para ES con destino RW son:", final_exports2)

# Total gross exports
total_exports1 = round(int_exports1 + final_exports1, 1)
print("Las exportaciones totales para ES con destino FR son:", total_exports1)

total_exports2 = round(int_exports2 + final_exports2, 1)
print("Las exportaciones totales para ES con destino RW son:", total_exports2)

Las exportaciones de productos intermedios para ES con destino FR son: 33893.7
Las exportaciones de productos intermedios para ES con destino RW son: 202059.8
Las exportaciones de productos terminados para ES con destino FR son: 21958.3
Las exportaciones de productos terminados para ES con destino RW son: 121021.1
Las exportaciones totales para ES con destino FR son: 55852.0
Las exportaciones totales para ES con destino RW son: 323080.9


In [181]:
# 4.5 Gross imports

# Intermediate products
int_imports1 = round(df_z_grtr.loc[(df_z_grtr['origen2'] == 'FR') & (df_z_grtr['destino2'] == 'ES'), 'obsValue2'].sum(), 1)
print("Las importaciones de productos intermedios para ES procedentes de FR son:", int_imports1)

int_imports2 = round(df_z_grtr.loc[(df_z_grtr['origen2'] == 'RW') & (df_z_grtr['destino2'] == 'ES'), 'obsValue2'].sum(), 1)
print("Las importaciones de productos intermedios para ES procedentes de RW son:", int_imports2)

# Final products
final_imports1 = round(df_y_grtr.loc[(df_y_grtr['origen2'] == 'FR') & (df_y_grtr['destino2'] == 'ES'), 'obsValue2'].sum(), 1)
print("Las importaciones de productos terminados para ES procedentes de FR son:", final_imports1)

final_imports2 = round(df_y_grtr.loc[(df_y_grtr['origen2'] == 'RW') & (df_y_grtr['destino2'] == 'ES'), 'obsValue2'].sum(), 1)
print("Las importaciones de productos terminados para ES procedentes de RW son:", final_imports2)

# Total gross imports
total_imports1 = round(int_imports1 + final_imports1, 1)
print("Las importaciones totales para ES procedentes de FR son:", total_imports1)
total_imports2 = round(int_imports2 + final_imports2, 1)
print("Las importaciones totales para ES procedentes de RW son:", total_imports2)

Las importaciones de productos intermedios para ES procedentes de FR son: 27561.3
Las importaciones de productos intermedios para ES procedentes de RW son: 220044.5
Las importaciones de productos terminados para ES procedentes de FR son: 12939.2
Las importaciones de productos terminados para ES procedentes de RW son: 117173.1
Las importaciones totales para ES procedentes de FR son: 40500.5
Las importaciones totales para ES procedentes de RW son: 337217.6


In [182]:
# 4.6 Gross trade balance

# Difference between Gross Exports and Gross Imports
gross_trade_balance1 = round(total_exports1 - total_imports1, 1)
print("El balance de exportaciones/importaciones de ES con FR son:", gross_trade_balance1)
gross_trade_balance2 = round(total_exports2 - total_imports2, 1)
print("El balance de exportaciones/importaciones de ES con RW son:", gross_trade_balance2)

El balance de exportaciones/importaciones de ES con FR son: 15351.5
El balance de exportaciones/importaciones de ES con RW son: -14136.7


In [183]:
# 4.7 Gross exports, partner shares (%) (ES-FR)

df_z_grtr1 = df_z_grtr.loc[(df_z_grtr['origen'].str.contains('ES')) & (df_z_grtr['destino2'] == 'FR')].reset_index(drop=True)
df_z_grtr1=pd.DataFrame(df_z_grtr1.groupby(['agrega2'])['obsValue2'].sum()).reset_index()
df_z_grtr1['gross exports %'] = (df_z_grtr1['obsValue2'] / int_exports1)*100
df_z_grtr1

,agrega2,obsValue2,gross exports %
0,ES_A*FR,1387.966,4.095056
1,ES_BDE*FR,799.750,2.359583
2,ES_C*FR,18769.316,55.377005
3,ES_F*FR,152.697,0.450517
4,ES_GHI*FR,9317.705,27.490964
5,ES_J*FR,522.999,1.543057
6,ES_K*FR,380.455,1.122495
7,ES_L*FR,25.788,0.076085
8,ES_MN*FR,2179.499,6.430396
9,ES_OPQ*FR,248.695,0.733750


In [184]:
# 4.7 Gross exports, partner shares (%) (ES-RW)

df_z_grtr2 = df_z_grtr.loc[(df_z_grtr['origen'].str.contains('ES')) & (df_z_grtr['destino2'] == 'RW')].reset_index(drop=True)
df_z_grtr2=pd.DataFrame(df_z_grtr2.groupby(['agrega2'])['obsValue2'].sum()).reset_index()
df_z_grtr2['gross exports %'] = (df_z_grtr2['obsValue2'] / int_exports2)*100
df_z_grtr2

,agrega2,obsValue2,gross exports %
0,ES_A*RW,9530.994,4.716917
1,ES_BDE*RW,5195.943,2.571488
2,ES_C*RW,102748.230,50.850407
3,ES_F*RW,1459.858,0.722488
4,ES_GHI*RW,52734.549,26.098486
5,ES_J*RW,7080.433,3.504127
6,ES_K*RW,4202.726,2.079942
7,ES_L*RW,161.760,0.080056
8,ES_MN*RW,16443.755,8.138064
9,ES_OPQ*RW,1886.723,0.933745


In [185]:
# 4.8 Gross imports, partner shares (%) (FR-ES)

df_z_grtr3 = df_z_grtr.loc[(df_z_grtr['origen'].str.contains('FR')) & (df_z_grtr['destino2'] == 'ES')].reset_index(drop=True)
df_z_grtr3=pd.DataFrame(df_z_grtr3.groupby(['agrega2'])['obsValue2'].sum()).reset_index()
df_z_grtr3['gross exports %'] = (df_z_grtr3['obsValue2'] / int_imports1)*100
df_z_grtr3

,agrega2,obsValue2,gross exports %
0,FR_A*ES,739.821,2.684275
1,FR_BDE*ES,1733.126,6.288259
2,FR_C*ES,14442.615,52.401792
3,FR_F*ES,0.045,0.000163
4,FR_GHI*ES,6272.589,22.758683
5,FR_J*ES,858.825,3.116054
6,FR_K*ES,353.814,1.283735
7,FR_L*ES,0.143,0.000519
8,FR_MN*ES,2840.751,10.307028
9,FR_OPQ*ES,296.203,1.074706


In [186]:
# 4.8 Gross imports, partner shares (%) (RW-ES)

df_z_grtr4 = df_z_grtr.loc[(df_z_grtr['origen'].str.contains('RW')) & (df_z_grtr['destino2'] == 'ES')].reset_index(drop=True)
df_z_grtr4=pd.DataFrame(df_z_grtr4.groupby(['agrega2'])['obsValue2'].sum()).reset_index()
df_z_grtr4['gross exports %'] = (df_z_grtr4['obsValue2'] / int_imports2)*100
df_z_grtr4

,agrega2,obsValue2,gross exports %
0,RW_A*ES,4144.088,1.883295
1,RW_BDE*ES,23958.424,10.887990
2,RW_C*ES,109707.330,49.856883
3,RW_F*ES,682.366,0.310104
4,RW_GHI*ES,44650.659,20.291650
5,RW_J*ES,7996.556,3.634063
6,RW_K*ES,12499.744,5.680553
7,RW_L*ES,314.938,0.143125
8,RW_MN*ES,10536.311,4.788264
9,RW_OPQ*ES,5234.553,2.378861


In [187]:
# 5.1 Domestic value added content of gross exports

# Filtrar el DataFrame por filas y columnas que contienen "ES"
df_inversa_ES = df_inversa.loc[df_inversa.index.str.contains('ES'), df_inversa.columns.str.contains('ES')]

# Filtrar el DataFrame por filas y columnas que contienen "ES"
df_ratio_ES = df_ratio.loc[df_ratio.index, df_ratio.columns.str.contains('ES')]

matriz_inversa_ES = df_inversa_ES.values
matriz_ratio_ES = df_ratio_ES.values

a = np.dot(matriz_ratio_ES, matriz_inversa_ES)
a_transpuesta = np.transpose(a)

In [188]:
df_zy_grtr_table_filtrado1 = df_zy_grtr_table.loc[df_zy_grtr_table.index.str.contains('ES'), df_zy_grtr_table.columns.str.contains('FR')]
matriz_export_ES_FR = df_zy_grtr_table_filtrado1.values

df_zy_grtr_table_filtrado2 = df_zy_grtr_table.loc[df_zy_grtr_table.index.str.contains('ES'), df_zy_grtr_table.columns.str.contains('RW')]
matriz_export_ES_RW = df_zy_grtr_table_filtrado2.values

In [189]:
exgr_dva_ES_FR = a_transpuesta * matriz_export_ES_FR
exgr_dva_ES_WR = a_transpuesta * matriz_export_ES_RW

In [190]:
exgr_dva_ES_FR

array([[ 3396.52278072],
       [  581.49954266],
       [19781.22151892],
       [  148.40771963],
       [ 9450.13525887],
       [ 1159.17909842],
       [  473.27465302],
       [   30.40146054],
       [ 2432.32587726],
       [  299.21579142],
       [  149.96720009]])

In [191]:
exgr_dva_ES_WR

array([[ 13424.45024118],
       [  3868.92506973],
       [102887.23544015],
       [  1832.00092361],
       [ 62813.45500013],
       [ 11651.08254773],
       [  5711.82600269],
       [   272.27931472],
       [ 19077.95614469],
       [  2612.99840673],
       [  1177.93001728]])

In [192]:
# 5.2 Domestic value added share of gross exports

suma_numerador = exgr_dva_ES_FR + exgr_dva_ES_WR
suma_denominador = matriz_export_ES_FR + matriz_export_ES_RW

exgr_dvash = (suma_numerador / suma_denominador) *100

In [193]:
# 5.3 Industry domestic value added contribution to gross exports

suma_numerador = exgr_dva_ES_FR + exgr_dva_ES_WR
suma_denominador = np.sum(matriz_export_ES_FR) + np.sum(matriz_export_ES_RW)

exgr_dtvaind = (suma_numerador / suma_denominador) *100

In [194]:
# 5.4 Domestic value added in gross exports, partner shares

numerador1 = exgr_dva_ES_FR
numerador2 = exgr_dva_ES_WR
suma_denominador = exgr_dva_ES_FR + exgr_dva_ES_WR

exgr_dvapsh1 = (numerador1 / suma_denominador) *100
exgr_dvapsh2 = (numerador2 / suma_denominador) *100

In [195]:
# 5.5 Direct domestic industry value added content of gross exports

diagonal = np.diag(matriz_inversa_ES)
nueva_matriz = np.diag(diagonal)
a =np.dot(matriz_ratio_ES, nueva_matriz)
a_transpuesta = np.transpose(a)

In [196]:
exgr_ddc = a_transpuesta * (matriz_export_ES_FR + matriz_export_ES_RW)
exgr_ddc

array([[11553.92399606],
       [ 2921.8903283 ],
       [64654.13979052],
       [ 1289.59690161],
       [53987.37978805],
       [ 8828.78340728],
       [ 4968.74454919],
       [  272.63265292],
       [17706.28711844],
       [ 2481.64701869],
       [ 1087.02528103]])

In [197]:
# 5.6 Indirect domestic value added content of gross exports (originating from domestic intermediates)

nueva_matriz = matriz_inversa_ES.copy()
np.fill_diagonal(nueva_matriz, 0)
a =np.dot(matriz_ratio_ES, nueva_matriz)
a_transpuesta = np.transpose(a)

In [198]:
exgr_idc = a_transpuesta * (matriz_export_ES_FR + matriz_export_ES_RW)
exgr_idc

array([[5.26704903e+03],
       [1.52853428e+03],
       [5.80143172e+04],
       [6.90811742e+02],
       [1.82762105e+04],
       [3.98147824e+03],
       [1.21635611e+03],
       [3.00481223e+01],
       [3.80399490e+03],
       [4.30567179e+02],
       [2.40871936e+02]])

In [199]:
exgr_dva_ES_FR + exgr_dva_ES_WR

array([[ 16820.9730219 ],
       [  4450.42461239],
       [122668.45695908],
       [  1980.40864323],
       [ 72263.590259  ],
       [ 12810.26164616],
       [  6185.10065571],
       [   302.68077525],
       [ 21510.28202195],
       [  2912.21419815],
       [  1327.89721737]])

In [200]:
exgr_ddc

array([[11553.92399606],
       [ 2921.8903283 ],
       [64654.13979052],
       [ 1289.59690161],
       [53987.37978805],
       [ 8828.78340728],
       [ 4968.74454919],
       [  272.63265292],
       [17706.28711844],
       [ 2481.64701869],
       [ 1087.02528103]])

In [201]:
exgr_idc

array([[5.26704903e+03],
       [1.52853428e+03],
       [5.80143172e+04],
       [6.90811742e+02],
       [1.82762105e+04],
       [3.98147824e+03],
       [1.21635611e+03],
       [3.00481223e+01],
       [3.80399490e+03],
       [4.30567179e+02],
       [2.40871936e+02]])

In [202]:
# 5.7 Reimported domestic value added content of gross exports

exgr_rim = (exgr_dva_ES_FR + exgr_dva_ES_WR) - exgr_ddc - exgr_idc
exgr_rim

array([[ 3.63797881e-12],
       [-6.82121026e-13],
       [ 0.00000000e+00],
       [ 4.54747351e-13],
       [ 1.09139364e-11],
       [-1.81898940e-12],
       [ 1.13686838e-12],
       [ 0.00000000e+00],
       [ 2.72848411e-12],
       [ 0.00000000e+00],
       [ 1.42108547e-13]])